In [53]:
import models
import data
from models import segmenter, tb_logs
from models.utils import loss_functions
import os.path as osp
import argparse
import csv
import sys
import os
import rasterio
from pytorch_lightning import Trainer
from torch.utils.data import Dataset, DataLoader
import itertools
import operator
from PIL import Image
import numpy as np

In [54]:
model_segmenter = "Unet"
model_encoder = "efficientnet-b2"
model_encoder_weights = "imagenet"  # Leave None if not using weights.
data_type = "npy"  # options are "npy" or "tiff"
epochs = 20
learning_rate = 1e-4
dataloader_workers = 12
validation_fraction = 0.2
batch_size = 64
log_step_frequency = 10
version = -1  # Keep -1 if loading the latest model version.
save_top_k_checkpoints = 3
loss_function = loss_functions.rmse_loss

sentinel_1_bands = {
    "VV ascending": 1,
    "VH ascending": 1,
    "VV descending": 1,
    "VH descending": 1
}

sentinel_2_bands = {
    "B2-Blue": 1,
    "B3-Green": 1,
    "B4-Red": 1,
    "B5-Veg red edge 1": 1,
    "B6-Veg red edge 2": 1,
    "B7-Veg red edge 3": 1,
    "B8-NIR": 1,
    "B8A-Narrow NIR": 1,
    "B11-SWIR 1": 1,
    "B12-SWIR 2": 1,
    "Cloud probability": 0
}

s1_list = list(sentinel_1_bands.values())
s2_list = list(sentinel_2_bands.values())

s1_bands_indicator = "S1-" + ''.join(str(x) for x in s1_list)
s2_bands_indicator = "S2-" + ''.join(str(x) for x in s2_list)

parser = argparse.ArgumentParser()

if model_encoder_weights is not None:
    model_identifier = f"{model_segmenter}_{model_encoder}_{model_encoder_weights}_{s1_bands_indicator}_{s2_bands_indicator}"
else:
    model_identifier = f"{model_segmenter}_{model_encoder}_{s1_bands_indicator}_{s2_bands_indicator}"

parser.add_argument('--model_identifier', default=model_identifier, type=str)
parser.add_argument('--segmenter_name', default=model_segmenter, type=str)
parser.add_argument('--encoder_name', default=model_encoder, type=str)
parser.add_argument('--encoder_weights', default=model_encoder_weights, type=str)
parser.add_argument('--model_version', default=version, type=int)
parser.add_argument('--data_type', default=data_type, type=str)

data_path = osp.dirname(data.__file__)
data_path = r"C:\Users\kuipe\Desktop\Epoch\forestbiomass\data"
models_path = osp.dirname(models.__file__)

# Note: Converted data does not have an explicit label path, as labels are stored within training_features
parser.add_argument('--converted_training_features_path', default=str(osp.join(data_path, "converted")), type=str)
parser.add_argument('--converted_testing_features_path', default=str(osp.join(data_path, "testing_converted")), type=str)

parser.add_argument('--tiff_training_features_path', default=str(osp.join(data_path, "imgs", "train_features")))
parser.add_argument('--tiff_training_labels_path', default=str(osp.join(data_path, "imgs", "train_agbm")))
parser.add_argument('--tiff_testing_features_path', default=str(osp.join(data_path, "imgs", "test_features")))

parser.add_argument('--training_ids_path', default=str(osp.join(data_path, "patch_names")), type=str)
parser.add_argument('--testing_ids_path', default=str(osp.join(data_path, "test_patch_names")), type=str)

parser.add_argument('--current_model_path', default=str(osp.join(models_path, "tb_logs", model_identifier)), type=str)
parser.add_argument('--submission_folder_path', default=str(osp.join(data_path, "imgs", "test_agbm")), type=str)

parser.add_argument('--dataloader_workers', default=dataloader_workers, type=int)
parser.add_argument('--batch_size', default=batch_size, type=int)
parser.add_argument('--epochs', default=epochs, type=int)
parser.add_argument('--learning_rate', default=learning_rate, type=float)
parser.add_argument('--validation_fraction', default=validation_fraction, type=float)
parser.add_argument('--log_step_frequency', default=log_step_frequency, type=int)
parser.add_argument('--save_top_k_checkpoints', default=save_top_k_checkpoints, type=int)

parser.add_argument('--S1_band_selection', default=s1_list, type=list)
parser.add_argument('--S2_band_selection', default=s2_list, type=list)
parser.add_argument('--loss_function', default=loss_function)

sys.argv = ['-f']
args = parser.parse_args()

In [55]:
model = segmenter.load_model(args)

Getting saved model...


In [56]:
with open(args.testing_ids_path, newline='') as f:
    reader = csv.reader(f)
    patch_name_data = list(reader)
chip_ids = patch_name_data[0]

print(len(chip_ids))

id_month_list = []

for id in chip_ids:
    for month in range(0, 12):
        month_patch_path = osp.join(args.converted_testing_features_path, id, str(month))
        if osp.exists(osp.join(month_patch_path, "S2")):
            id_month_list.append((id, str(month)))

print(len(id_month_list))

new_dataset = segmenter.SubmissionDatasetMakerConverted(args.converted_testing_features_path,
                                              id_month_list,
                                              args.S1_band_selection,
                                              args.S2_band_selection)


trainer = Trainer(accelerator="gpu", devices=1)

dl = DataLoader(new_dataset, num_workers=12)

predictions = trainer.predict(model, dataloaders=dl)

transformed_predictions = [x.cpu().squeeze().detach().numpy() for x in predictions]

linked_tensor_list = list(zip(id_month_list, transformed_predictions))

2773


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: C:\Users\Lars\PycharmProjects\forestbiomass\analysis\exploratory_data_analysis\lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


30072


Predicting: 0it [00:00, ?it/s]

In [67]:
#linked_tensor_list = [(a,b,c) for ((a,b),c) in linked_tensor_list]

linked_tensor_list = sorted(linked_tensor_list, key=operator.itemgetter(0))

it = itertools.groupby(linked_tensor_list, operator.itemgetter(0))

selection = [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]

for key, subiter in it:

    selected_tensors = []

    group_list = list(subiter)

    for x in group_list:

        if selection[int(x[1])] == 1:
            selected_tensors.append(x[2])
            print(x[2])

    final_result = sum(selected_tensors) / len(selected_tensors)

    print(final_result)
    break

    #agbm_path = osp.join(r"C:\Users\Lars\PycharmProjects\forestbiomass\data\imgs\test_agbm", f"{x[0]}_agbm.tif")

    #im = Image.fromarray(final_result)
    #im.save(agbm_path)

[[ 59.029915   79.5891     72.540764  ...   2.8861406   4.540174
    5.133126 ]
 [ 65.209076   74.39052    59.145863  ...   1.7179681   2.6828048
    3.0594013]
 [ 54.30711    54.618656   41.026386  ...   1.319069    1.6412218
    2.1723707]
 ...
 [ 30.460993   30.962322   19.267084  ...  67.026276   93.08416
   93.4413   ]
 [ 36.816048   38.501015   20.487526  ...  93.540344  120.18859
  110.06517  ]
 [ 33.860912   40.214436   31.474655  ...  92.99066   112.91371
   97.568344 ]]
[[ 49.760834   69.112114   65.844444  ...   2.77302     3.610797
    3.62935  ]
 [ 49.44073    58.44365    55.187466  ...   2.3489065   2.7964272
    2.600866 ]
 [ 33.85662    37.45817    48.13125   ...   0.9019605   1.23821
    1.9641405]
 ...
 [ 42.417397   45.36534    35.12472   ... 108.439835  118.55844
  101.330246 ]
 [ 37.145863   38.31481    22.982264  ... 120.34524   131.29677
  107.743965 ]
 [ 31.565638   39.13135    36.187912  ... 118.500336  119.96414
   95.74262  ]]
[[ 57.510597   77.280174   73.50

In [44]:
linked_tensor_list = sorted(linked_tensor_list, key=operator.itemgetter(0))

it = itertools.groupby(linked_tensor_list, operator.itemgetter(0))

month_differences_relative = [[], [], [], [], [], [], [], [], [], [], [], []]
month_differences_total = [[], [], [], [], [], [], [], [], [], [], [], []]

for key, subiter in it:
    group_list = list(subiter)

    label = np.load(osp.join(args.converted_training_features_path, key, "label.npy"), allow_pickle=True)

    group_list = [(a, b, np.sqrt(np.mean((c-label)**2))) for (a, b, c) in group_list]

    for x in group_list:
        month_differences_total[int(x[1])].append(x[2])

    average = sum([c for (a, b, c) in group_list]) / len(group_list)

    for x in group_list:
        month_differences_relative[int(x[1])].append(x[2] / average)


In [49]:

for index, month in enumerate(month_differences_relative):
    print(index, sum(month) / len(month))
print()
for index, month in enumerate(month_differences_total):
    print(index, sum(month) / len(month))

0 1.0105151507110341
1 1.0325267808872771
2 1.0434550634288864
3 1.0912142840990064
4 1.090666266270216
5 0.9817669535379383
6 0.9826462290135376
7 0.9846283583147567
8 0.9787239365395931
9 0.9748756408282437
10 0.9711862288588636
11 0.9888754695074633

0 34.89952747203171
1 35.7370954440217
2 38.18222205503726
3 51.43772997349191
4 41.691278596544755
5 33.93907626540598
6 33.605420633419214
7 33.36657497237749
8 32.931449264799
9 32.577749475015295
10 32.634411447601636
11 33.313490389625144


In [51]:
print(linked_tensor_list[0])

('0003d2eb', '0', array([[11.71017  , 12.8406105,  6.262487 , ..., 15.377561 , 23.88218  ,
        23.153402 ],
       [10.321426 ,  9.108254 ,  3.2345195, ..., 11.245694 , 18.986055 ,
        19.108906 ],
       [ 4.789643 ,  5.79979  ,  4.3253355, ..., 11.153532 , 14.229192 ,
        13.6119795],
       ...,
       [22.327866 , 25.537874 , 18.560093 , ..., 16.106602 , 21.410442 ,
        20.889202 ],
       [22.721504 , 24.741869 , 15.772408 , ..., 17.249743 , 19.898855 ,
        17.892534 ],
       [18.146606 , 20.73674  , 17.445465 , ..., 20.568035 , 18.813993 ,
        15.410244 ]], dtype=float32))
